In [1]:
#모듈 임포트
import tensorflow as tf

## 데이터 로드,전처리

In [2]:
(x_train,y_train),(x_valid,y_valid) = tf.keras.datasets.mnist.load_data()

x_train = x_train[...,tf.newaxis]/255.0
x_valid = x_valid[...,tf.newaxis]/255.0

train_data = tf.data.Dataset.from_tensor_slices((x_train,y_train)).shuffle(1000).batch(128)
valid_data = tf.data.Dataset.from_tensor_slices((x_valid,y_valid)).batch(128)

## 모델링

In [3]:
class MyModel(tf.keras.models.Model):
    def __init__(self):
        super(MyModel,self).__init__()
        self.cnn1 = tf.keras.layers.Conv2D(32,(3,3),padding='same',activation='relu')
        self.maxpool1 = tf.keras.layers.MaxPool2D(2,2)
        self.cnn2 = tf.keras.layers.Conv2D(64,(3,3),padding='same',activation='relu')
        self.maxpool2 = tf.keras.layers.MaxPool2D(2,2)
        self.cnn3 = tf.keras.layers.Conv2D(64,(3,3),padding='same',activation='relu')
        self.flatten = tf.keras.layers.Flatten()
        self.Dense1 = tf.keras.layers.Dense(32,activation='relu')
        self.output_ = tf.keras.layers.Dense(10,activation='softmax')
    
    def call(self,input_):
        x = self.cnn1(input_)
        x = self.maxpool1(x)
        x = self.cnn2(x)
        x = self.maxpool2(x)
        x = self.cnn3(x)
        x = self.flatten(x)
        x = self.Dense1(x)
        x = self.output_(x)
        return x

In [4]:
model = MyModel()

input_ = tf.keras.layers.Input(shape = (28,28,1))
model(input_)

model.summary()

Model: "my_model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              multiple                  320       
_________________________________________________________________
max_pooling2d (MaxPooling2D) multiple                  0         
_________________________________________________________________
conv2d_1 (Conv2D)            multiple                  18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 multiple                  0         
_________________________________________________________________
conv2d_2 (Conv2D)            multiple                  36928     
_________________________________________________________________
flatten (Flatten)            multiple                  0         
_________________________________________________________________
dense (Dense)                multiple                  100

## CheckPoint

In [5]:
checkpoint_path = 'my_checkpoint.ckpt'
checkpoint = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                save_weights_only=True, # 가중치만 저장
                                                save_best_only=True,    # 기준값이 개선 되었을때만 저장
                                                monitor='val_loss',     # 기준값: val_loss
                                                verbose=1)

## 컴파일,학습

In [6]:
model.compile(optimizer = 'adam', loss = 'sparse_categorical_crossentropy',metrics = ['acc'])

In [7]:
model.fit(train_data,
         validation_data = (valid_data),
         epochs = 10,
          callbacks=[checkpoint]
         )

Epoch 1/10
469/469 [==============================] - 23s 48ms/step - loss: 0.5278 - acc: 0.8407 - val_loss: 0.0693 - val_acc: 0.9764

Epoch 00001: val_loss improved from inf to 0.06935, saving model to my_checkpoint.ckpt
Epoch 2/10
469/469 [==============================] - 23s 49ms/step - loss: 0.0644 - acc: 0.9811 - val_loss: 0.0472 - val_acc: 0.9845

Epoch 00002: val_loss improved from 0.06935 to 0.04723, saving model to my_checkpoint.ckpt
Epoch 3/10
469/469 [==============================] - 23s 48ms/step - loss: 0.0430 - acc: 0.9869 - val_loss: 0.0348 - val_acc: 0.9877

Epoch 00003: val_loss improved from 0.04723 to 0.03481, saving model to my_checkpoint.ckpt
Epoch 4/10
469/469 [==============================] - 21s 44ms/step - loss: 0.0326 - acc: 0.9900 - val_loss: 0.0370 - val_acc: 0.9870

Epoch 00004: val_loss did not improve from 0.03481
Epoch 5/10
469/469 [==============================] - 22s 47ms/step - loss: 0.0266 - acc: 0.9919 - val_loss: 0.0298 - val_acc: 0.9907

Epoch

## 가중치 적용

In [8]:
model.load_weights(checkpoint_path)